<a href="https://colab.research.google.com/github/digitalarkpool/CrewAI_Edify/blob/main/CrewAI_Audit_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prep for meeting using Exa.ai search https://exa.ai/

In [ ]:
!pip -q install crewai duckduckgo-search
!pip -q install 'crewai[tools]' decouple langchain-exa exa_py==1.0.7

# crewai==0.22.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip show crewai

Name: crewai
Version: 0.22.5
Summary: Cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.
Home-page: 
Author: Joao Moura
Author-email: joao@crewai.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: click, instructor, langchain, langchain-openai, openai, opentelemetry-api, opentelemetry-exporter-otlp-proto-http, opentelemetry-sdk, pydantic, python-dotenv, regex
Required-by: 


In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["EXA_API_KEY"] = userdata.get('EXA_API_KEY')

## Tool

In [ ]:
from exa_py import Exa
from langchain.agents import tool

class ExaSearchTool:
	@tool
	def search(query: str):
		"""Search for a webpage based on the query."""
		return ExaSearchTool._exa().search(f"{query}", use_autoprompt=True, num_results=3)

	@tool
	def find_similar(url: str):
		"""Search for webpages similar to a given URL.
		The url passed in should be a URL returned from `search`.
		"""
		return ExaSearchTool._exa().find_similar(url, num_results=3)

	@tool
	def get_contents(ids: str):
		"""Get the contents of a webpage.
		The ids must be passed in as a list, a list of ids returned from `search`.
		"""
		ids = eval(ids)
		contents = str(ExaSearchTool._exa().get_contents(ids))
		print(contents)
		contents = contents.split("URL:")
		contents = [content[:1000] for content in contents]
		return "\n\n".join(contents)

	def tools():
		return [ExaSearchTool.search, ExaSearchTool.find_similar, ExaSearchTool.get_contents]

	def _exa():
		return Exa(api_key=os.environ["EXA_API_KEY"])

## Agents

In [ ]:
from textwrap import dedent
from crewai import Agent


class MeetingPreparationAgents():
	def research_agent(self):
		return Agent(
			role='Research Specialist',
			goal='Conduct thorough research on people and companies involved in the meeting',
			tools=ExaSearchTool.tools(),
			backstory=dedent("""\
					As a Research Specialist, your mission is to uncover detailed information
					about the individuals and entities participating in the meeting. Your insights
					will lay the groundwork for strategic meeting preparation."""),
			verbose=True
		)

	def industry_analysis_agent(self):
		return Agent(
			role='Industry Analyst',
			goal='Analyze the current industry trends, challenges, and opportunities',
			tools=ExaSearchTool.tools(),
			backstory=dedent("""\
					As an Industry Analyst, your analysis will identify key trends,
					challenges facing the industry, and potential opportunities that
					could be leveraged during the meeting for strategic advantage."""),
			verbose=True
		)

	def meeting_strategy_agent(self):
		return Agent(
			role='Meeting Strategy Advisor',
			goal='Develop talking points, questions, and strategic angles for the meeting',
			tools=ExaSearchTool.tools(),
			backstory=dedent("""\
					As a Strategy Advisor, your expertise will guide the development of
					talking points, insightful questions, and strategic angles
					to ensure the meeting's objectives are achieved."""),
			verbose=True
		)

	def summary_and_briefing_agent(self):
		return Agent(
			role='Briefing Coordinator',
			goal='Compile all gathered information into a concise, informative briefing document',
			tools=ExaSearchTool.tools(),
			backstory=dedent("""\
					As the Briefing Coordinator, your role is to consolidate the research,
					analysis, and strategic insights."""),
			verbose=True
		)

## Tasks

In [ ]:
from crewai import Task

class MeetingPreparationTasks():
	def research_task(self, agent, participants, context):
		return Task(
			description=dedent(f"""\
				Conduct comprehensive research on each of the individuals and companies
				involved in the upcoming meeting. Gather information on recent
				news, achievements, professional background, and any relevant
				business activities.

				Participants: {participants}
				Meeting Context: {context}"""),
			expected_output=dedent("""\
				A detailed report summarizing key findings about each participant
				and company, highlighting information that could be relevant for the meeting."""),
			async_execution=True,
			agent=agent
		)

	def industry_analysis_task(self, agent, participants, context):
		return Task(
			description=dedent(f"""\
				Analyze the current industry trends, challenges, and opportunities
				relevant to the meeting's context. Consider market reports, recent
				developments, and expert opinions to provide a comprehensive
				overview of the industry landscape.

				Participants: {participants}
				Meeting Context: {context}"""),
			expected_output=dedent("""\
				An insightful analysis that identifies major trends, potential
				challenges, and strategic opportunities."""),
			async_execution=True,
			agent=agent
		)

	def meeting_strategy_task(self, agent, context, objective):
		return Task(
			description=dedent(f"""\
				Develop strategic talking points, questions, and discussion angles
				for the meeting based on the research and industry analysis conducted

				Meeting Context: {context}
				Meeting Objective: {objective}"""),
			expected_output=dedent("""\
				Complete report with a list of key talking points, strategic questions
				to ask to help achieve the meetings objective during the meeting."""),
			agent=agent
		)

	def summary_and_briefing_task(self, agent, context, objective):
		return Task(
			description=dedent(f"""\
				Compile all the research findings, industry analysis, and strategic
				talking points into a concise, comprehensive briefing document for
				the meeting.
				Ensure the briefing is easy to digest and equips the meeting
				participants with all necessary information and strategies.

				Meeting Context: {context}
				Meeting Objective: {objective}"""),
			expected_output=dedent("""\
				A well-structured briefing document that includes sections for
				participant bios, industry overview, talking points, and
				strategic recommendations."""),
			agent=agent
		)

## Crew

In [ ]:
from crewai import Crew


tasks = MeetingPreparationTasks()
agents = MeetingPreparationAgents()

print("## Welcome to the Meeting Prep Crew")
print('-------------------------------')
participants = input("What are the emails for the participants (other than you) in the meeting?\n")
context = input("What is the context of the meeting?\n")
objective = input("What is your objective for this meeting?\n")


## Welcome to the Meeting Prep Crew
-------------------------------
What are the emails for the participants (other than you) in the meeting?
jay@jayz.com
What is the context of the meeting?
Creating a beats alternative
What is your objective for this meeting?
convince him to partner with us


In [ ]:
# Create Agents
researcher_agent = agents.research_agent()
industry_analyst_agent = agents.industry_analysis_agent()
meeting_strategy_agent = agents.meeting_strategy_agent()
summary_and_briefing_agent = agents.summary_and_briefing_agent()

# Create Tasks
research = tasks.research_task(researcher_agent, participants, context)
industry_analysis = tasks.industry_analysis_task(industry_analyst_agent, participants, context)
meeting_strategy = tasks.meeting_strategy_task(meeting_strategy_agent, context, objective)
summary_and_briefing = tasks.summary_and_briefing_task(summary_and_briefing_agent, context, objective)

meeting_strategy.context = [research, industry_analysis]
summary_and_briefing.context = [research, industry_analysis, meeting_strategy]


In [ ]:


# Create Crew responsible for Copy
crew = Crew(
	agents=[
		researcher_agent,
		industry_analyst_agent,
		meeting_strategy_agent,
		summary_and_briefing_agent
	],
	tasks=[
		research,
		industry_analysis,
		meeting_strategy,
		summary_and_briefing
	]
)



In [ ]:
result = crew.kickoff()


# Print results
print("\n\n################################################")
print("## Here is the result")
print("################################################\n")
print(result)



> Entering new CrewAgentExecutor chain...


> Entering new CrewAgentExecutor chain...
I should start by searching for detailed information about the main participant of the meeting, Jay Z. This will include looking for his professional background, recent news, achievements, and relevant business activities.

Action: 
search

Action Input: 
{
  "query": "Jay Z professional background and recent news"
}
 

Title: Why Jay-Z has 5 careers, and you don’t
URL: https://www.marketwatch.com/story/why-jay-z-has-5-careers-and-you-dont-2013-04-05
ID: aKO8HNnRr1LAQOY6-LzHCg
Score: 0.18464410305023193
Published Date: 2013-04-05
Author: Jonnelle Marte
Text: None
Highlights: None
Highlight Scores: None


Title: Poetic justice: how Jay-Z became a civil rights champion
URL: https://www.theguardian.com/music/2019/mar/12/how-jay-z-became-a-civil-rights-champion
ID: LuX2vnBZdbeMnkO1ievlkQ
Score: 0.1781911849975586
Published Date: 2019-03-12
Author: Khushbu Shah
Text: None
Highlights: None
Highlight Score

In [ ]:
from IPython.display import display, Markdown
display(Markdown(result))


**Briefing Document: Creating a Beats Alternative**

**Participant Bio:**
Jay Z (Shawn Corey Carter) is a highly influential figure in the music industry. He has sold over 50 million albums and 75 million singles worldwide, making him one of the best-selling musicians of all time. Jay Z is the owner of the 40/40 Club, an upscale sports bar that started in New York City and has since expanded to Atlantic City and Chicago. He also co-owns the New York 40/40 Club and Roc Nation, an entertainment company that includes a record label, talent agency, touring and concert production company, music, film, and television production company, and music publishing house. Jay Z is known for his philanthropic work, he has established the Shawn Carter Foundation to help individuals facing socio-economic hardships further their education at post-secondary institutions. In 2015, he donated $1.5 million to Black Lives Matter.

**Industry Overview:**
The music streaming industry is growing, with streaming being the major driver of growth. In 2022, the US recorded music revenues grew 6% to a record high $15.9 billion, with streaming growing 7% to $13.3 billion, comprising 84% of revenues in 2022. On a global scale, the recorded music market grew by 18.5% in 2021, with paid subscription streaming revenues increasing by 21.9% to US$12.3 billion. There were 523 million users of paid subscription accounts at the end of 2021.

However, there are challenges in the industry. Music streaming and the attention recession have turned music into a background activity, with artists struggling to cut through the noise. Streaming is neither a place to build deeper relationships with fans nor to generate meaningful income. This presents a challenge for artists and industry players.

On the other hand, there are opportunities in the industry. Music's key audiences are seeking active, participatory experiences with music. These factors are catalysing the emergence of a new, counterculture music business that hinges on the artist-fan relationship, and the platforms that enable them. This suggests an opportunity to create platforms that foster deeper artist-fan relationships.

**Talking Points:**
1. Highlight the growth and potential of the music streaming industry, emphasizing the increase in revenues and the number of users of paid subscriptions.
2. Discuss the challenges in the music industry, such as the attention recession and the difficulty for artists to build deeper relationships with fans and generate meaningful income.
3. Present the opportunities in the industry for active, participatory experiences with music and the emergence of a new, counterculture music business that hinges on the artist-fan relationship.
4. Discuss the idea of a beats alternative that nurtures and empowers artists and fans, allowing them to participate in the creation of music.
5. Highlight Jay Z's diverse career, entrepreneurial spirit, and involvement in social activism, and how these could contribute to the creation of a beats alternative.

**Strategic Recommendations:**
1. Leverage Jay Z's influence and experience in the music industry, as well as his entrepreneurial spirit, to brainstorm and strategize for the creation of a beats alternative.
2. Consider the current challenges and opportunities in the music streaming industry when formulating strategies for the beats alternative.
3. Explore ways to foster deeper artist-fan relationships through the beats alternative.
4. Discuss how the beats alternative can contribute to social cause-related initiatives, given Jay Z's role as a civil rights champion.
5. Consider how Jay Z's recent activities and ventures can influence and contribute to the beats alternative.